In [12]:
import pandas as pd
import nltk
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [13]:
filename = "nursery_rhymes.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
raw_text

'the queen _of_ hearts\n\n\n    the queen of hearts she made some tarts,\n      all on a summer\'s day;\n\n    the knave of hearts he stole those tarts,\n      and took them clean away.\n\n    the king of hearts called for those tarts,\n    and beat the knave full sore.\n\n    the knave of hearts brought back those tarts,\n      and vowed he\'d steal no more.\n\n\n\n\nsaint swithin\'s day\n\n\n    st. swithin\'s day, if thou dost rain,\n    for forty days it will remain;\n    st. swithin\'s day, if thou be fair,\n    for forty days \'t will rain no more.\n\n\n\n\ndance to your daddie\n\n\n      dance to your daddie,\n      my bonnie laddie,\n    dance to your daddie, my bonnie lamb!\n      you shall get a fishie\n      on a little dishie,\n    you shall get a fishie when the boat comes hame!\n\n      dance to your daddie,\n      my bonnie laddie,\n    dance to your daddie, and to your mammie sing!\n      you shall get a coatie,\n      and a pair of breekies,\n    you shall get a coatie

In [14]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)

In [15]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

In [17]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [18]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [21]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, y, epochs=5, batch_size=128, callbacks=callbacks_list)

Epoch 1/5
804/804 [==============================] - ETA: 0s - loss: 2.6650
Epoch 00001: loss improved from inf to 2.66498, saving model to weights-improvement-01-2.6650.hdf5
804/804 [==============================] - 296s 368ms/step - loss: 2.6650
Epoch 2/5
804/804 [==============================] - ETA: 0s - loss: 2.5256
Epoch 00002: loss improved from 2.66498 to 2.52558, saving model to weights-improvement-02-2.5256.hdf5
804/804 [==============================] - 309s 384ms/step - loss: 2.5256
Epoch 3/5
804/804 [==============================] - ETA: 0s - loss: 2.4440
Epoch 00003: loss improved from 2.52558 to 2.44401, saving model to weights-improvement-03-2.4440.hdf5
804/804 [==============================] - 327s 407ms/step - loss: 2.4440
Epoch 4/5
804/804 [==============================] - ETA: 0s - loss: 2.3989
Epoch 00004: loss improved from 2.44401 to 2.39894, saving model to weights-improvement-04-2.3989.hdf5
804/804 [==============================] - 329s 409ms/step - loss: